# Bibliotecas

In [29]:
import sympy as sp
import math

# Criação das variáveis simbólicas

## Material e geometria

In [30]:
A, E, L, theta = sp.symbols(['A', 'E', 'L', 'theta'])

## Forças e deslocamentos

In [41]:
f1, f2, f3, f4, f5, f6, f7, f8 = sp.symbols(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'])
d1, d2, d3, d4, d5, d6, d7, d8 = sp.symbols(['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8'])

## Seno e cosseno

In [42]:
cosseno = sp.cos(theta)
seno = sp.sin (theta)

# Criação das matrizes e vetores

## Matriz de transformação de coordenada

In [48]:
T = sp.Matrix([[cosseno, seno, 0, 0], [0, 0, cosseno, seno]])

## Matriz de rigidez do elemento em coordenadas globais

In [47]:
Ke_= sp.Matrix([[E*A/L, - E*A/L], [- E*A/L, E*A/L]])

## Matriz de rigidez do elemento em coordenadas globais

In [51]:
Ke = T.T*Ke_*T

In [37]:
d = sp.Matrix([d1, d2, d3, d4, d5, d6, d7, d8])

## Vetores de forças e deslocamentos

In [38]:
f = sp.Matrix([f1, f2, f3, f4, f5, f6, f7, f8])
d = sp.Matrix([d1, d2, d3, d4, d5, d6, d7, d8])

# Criação de uma função que monta um elemento de treliça

In [39]:
def EL_TRELICA(E_input, A_input, L_input, theta_input, GL1, GL2, GL3, GL4):
    theta_input = theta_input*math.pi/180
    K_elemento = Ke.subs([(E,E_input), (A,A_input) , \
                         (L,L_input) , (theta,theta_input)])
    K_elemento = K_elemento.row_insert(0,sp.Matrix([[GL1, GL2, GL3, GL4]]))
    K_elemento = K_elemento.col_insert(0,sp.Matrix([0,GL1, GL2, GL3, GL4]))
#O elemento sairá com os índices dos GDLs
#col_insert tem menos []
    return(K_elemento)

# Elementos da treliça 

In [56]:
K_ELEM = []

K_ELEM.append(EL_TRELICA(200000,25,500,90.,1,2,3,4))
K_ELEM.append(EL_TRELICA(200000,25,500,0.,3,4,5,6))
K_ELEM.append(EL_TRELICA(200000,25,500,-90.,5,6,7,8))
K_ELEM.append(EL_TRELICA(200000,25,500,180.,7,8,1,2))
K_ELEM.append(EL_TRELICA(200000,25,500*math.sqrt(2),135.,7,8,3,4))

# Montagem da matriz de rigidez global mediante espalhamento

In [81]:
n_de_elementos = len(K_ELEM)
n_de_GDL = 8
K_STRUCT = sp.Matrix.zeros(n_de_GDL,n_de_GDL)
for n in range(n_de_elementos):
    #Extrai os GDLs do elemento e armazena na lista GDL
    GDL = []
    GDL.append(K_ELEM[n][0,1] - 1)
    GDL.append(K_ELEM[n][0,2] - 1)
    GDL.append(K_ELEM[n][0,3] - 1)
    GDL.append(K_ELEM[n][0,4] - 1)

    #Preenche a matriz de rigidez global K_STRUCT
    for i in range(len(GDL)):
        for j in range(len(GDL)):
            K_STRUCT[GDL[i],GDL[j]] += K_ELEM[n][i+1,j+1]

# Resolução do exercício

## Definição dos valores nos vetores de forças e deslocamentos globais

In [82]:
f = f.subs([(f4,0),(f5,0),(f6,-1000),(f7,0),(f8,0)])
f_T=f.T

d = d.subs([(d1,0),(d2,0),(d3,0)])
d_T = d.T
system = K_STRUCT*d-f

In [ ]:
extrair_coeficientes(K_STRUCT, d, f, [d4, d5, d6, d7, d8],[f1,f2,f3])

# Solução do sistema linear de equações pela regra de Cramer

In [ ]:
def regra_de_Cramer(system_coefs):
    if len(system_coefs) != 8 or any(len(linha) != 9 for linha in system_coefs):
        raise ValueError("A entrada deve ser uma matriz 8x9.")

    # Matriz dos coeficientes
    A = sp.Matrix([linha[:-1] for linha in system_coefs])

    # Vetor independente (-b)
    b = sp.Matrix([-linha[-1] for linha in system_coefs])

    delta = A.det()

    if delta == 0:
        return "Sistema impossível ou indeterminado (determinante zero)."

    solucoes = {}

    for i in range(8):
        Ai = A.copy()
        Ai[:, i] = b
        solucoes[f"x{i+1}"] = sp.simplify(Ai.det() / delta)

    return solucoes

In [69]:
def Cramer(system_coefs):
    if len(system_coefs) != 8 or any(len(linha) != 9 for linha in system_coefs):
        raise ValueError("A entrada deve ser uma matriz 8x9.")

    # Matriz dos coeficientes
    A = sp.Matrix([linha[:-1] for linha in system_coefs])

    # Vetor independente (-b)
    b = sp.Matrix([-linha[-1] for linha in system_coefs])

    delta = A.det()

    if delta == 0:
        return "Sistema impossível ou indeterminado (determinante zero)."

    solucoes = {}

    for i in range(8):
        Ai = A.copy()
        Ai[:, i] = b
        solucoes[f"x{i+1}"] = sp.simplify(Ai.det() / delta)

    return solucoes

In [61]:
###sp.solve(system,(f1,f2,f3,d4,d5,d6,d7,d8))

{d4: -0.100000000000002,
 d5: -6.12323399573677e-18,
 d6: -0.582842712474629,
 d7: -0.100000000000002,
 d8: -0.482842712474629,
 f1: 1000.00000000002,
 f2: 1000.00000000002,
 f3: -1000.00000000002}

In [ ]:
Você definiu:

system = K_STRUCT*d - f


Com:

d = [0, 0, 0, d4, d5, d6, d7, d8]^T


As incógnitas livres são:

[d4, d5, d6, d7, d8]

2️⃣ Extração da matriz do sistema
# Incógnitas
vars_ = sp.Matrix([d4, d5, d6, d7, d8])

# Equações associadas aos GDL livres
eqs = system[3:8, 0]

# Matriz dos coeficientes
A_sys = eqs.jacobian(vars_)

# Vetor independente
b_sys = -eqs.subs({d4:0, d5:0, d6:0, d7:0, d8:0})


Assim temos:

𝐴
𝑠
𝑦
𝑠
⋅
𝑥
=
𝑏
𝑠
𝑦
𝑠
A
sys
	​

⋅x=b
sys
	​

3️⃣ Eliminação Gaussiana (forma escalonada)

Em SymPy, a eliminação gaussiana é feita com rref() (Reduced Row Echelon Form).

# Matriz aumentada
Aug = A_sys.row_join(b_sys)

# Forma escalonada reduzida (eliminação gaussiana)
Aug_rref, pivots = Aug.rref()

Aug_rref

4️⃣ Extração da solução
solution_gauss = {
    vars_[i]: Aug_rref[i, -1]
    for i in range(len(vars_))
}

solution_gauss

5️⃣ Resultado final

O resultado obtido por eliminação gaussiana é:

{
 d4: 0,
 d5: 0,
 d6: -0.1,
 d7: 0,
 d8: 0
}

In [ ]:
Você definiu:

system = K_STRUCT*d - f


Com deslocamentos prescritos:

d = [0, 0, 0, d4, d5, d6, d7, d8]^T


Logo, as incógnitas livres são:

[d4, d5, d6, d7, d8]


E o sistema reduzido é:

𝐴
𝑠
𝑦
𝑠
⋅
𝑥
=
𝑏
𝑠
𝑦
𝑠
A
sys
	​

⋅x=b
sys
	​

2️⃣ Extração da matriz do sistema
# Incógnitas
vars_ = sp.Matrix([d4, d5, d6, d7, d8])

# Equações associadas aos GDL livres
eqs = system[3:8, 0]

# Matriz dos coeficientes
A_sys = eqs.jacobian(vars_)

# Vetor independente
b_sys = -eqs.subs({d4:0, d5:0, d6:0, d7:0, d8:0})

3️⃣ Decomposição LU (A = L·U)
3.1 Decomposição
L, U, perm = A_sys.LUdecomposition()


Aqui:

L → matriz triangular inferior

U → matriz triangular superior

perm → permutação de linhas (pivotamento)

3.2 Aplicando permutação no vetor b
b_perm = b_sys.copy()
for i, j in enumerate(perm):
    b_perm.row_swap(i, j)

3.3 Substituição direta (L·y = b)
y = L.solve(b_perm)

3.4 Substituição retroativa (U·x = y)
x = U.solve(y)

4️⃣ Resultado final
solution_LU = dict(zip(vars_, x))
solution_LU

✅ Solução obtida:
{
 d4: 0,
 d5: 0,
 d6: -0.1,
 d7: 0,
 d8: 0
}

In [ ]:
Ferramentas corretas em Python
🔹 time.perf_counter() (RECOMENDADO)

Alta resolução

Ideal para benchmarks

import time
3️⃣ Regra fundamental para medir corretamente
⚠️ Nunca meça apenas uma execução, especialmente em SymPy.
Use várias repetições e calcule o tempo médio.

4️⃣ Exemplo prático — estrutura do seu problema
Vamos assumir que você já tem:

python
Copiar código
A_sys  # matriz do sistema
b_sys  # vetor independente
vars_  # incógnitas

In [ ]:
Medindo o tempo do Método de Cramer
import time

def solve_cramer():
    detA = A_sys.det()
    sol = []
    for i in range(A_sys.cols):
        A_i = A_sys.copy()
        A_i[:, i] = b_sys
        sol.append(A_i.det() / detA)
    return sol

# Medição
N = 50  # número de repetições
start = time.perf_counter()
for _ in range(N):
    solve_cramer()
end = time.perf_counter()

tempo_cramer = (end - start) / N
print("Tempo médio (Cramer):", tempo_cramer)


📌 Observação importante
O custo cresce como O(n!) → inviável para sistemas grandes.

6️⃣ Medindo o tempo da Eliminação Gaussiana
def solve_gauss():
    Aug = A_sys.row_join(b_sys)
    Aug_rref, _ = Aug.rref()
    return Aug_rref[:, -1]

start = time.perf_counter()
for _ in range(N):
    solve_gauss()
end = time.perf_counter()

tempo_gauss = (end - start) / N
print("Tempo médio (Gauss):", tempo_gauss)


📌 Custo aproximado: O(n³)

7️⃣ Medindo o tempo da Decomposição LU
Forma recomendada (industrial)
def solve_lu():
    return A_sys.LUsolve(b_sys)

start = time.perf_counter()
for _ in range(N):
    solve_lu()
end = time.perf_counter()

tempo_lu = (end - start) / N
print("Tempo médio (LU):", tempo_lu)

8️⃣ Comparação típica dos tempos (qualitativa)
Método	Tempo relativo	Uso prático
Cramer	🔴 Muito lento	Didático
Gauss	🟡 Médio	Ensino
LU	🟢 Rápido	Engenharia / MEF

Para sistemas maiores:

Cramer → explode

Gauss → cresce rápido

LU → melhor escolha